# Run training of CLRS algorithm

Copied main function from clrs_train.py. Helper functions are located in clrs_train_funcs.py

In [1]:
import functools
import os
import shutil
from typing import Any, Dict, List

import logging
import clrs
import jax
import jax.numpy as jnp
import numpy as np
import requests
import tensorflow as tf
import haiku as hk

import model

import flags
from clrs_train_funcs import *

2023-03-27 16:36:51.596749: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-03-27 16:36:51.596882: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-03-27 16:36:51.596888: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
!nvidia-smi

Mon Mar 27 16:36:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.65       Driver Version: 527.56       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8    13W / 139W |      0MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
FLAGS = flags.FLAGS

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
if FLAGS.hint_mode == 'encoded_decoded':
    encode_hints = True
    decode_hints = True
elif FLAGS.hint_mode == 'decoded_only':
    encode_hints = False
    decode_hints = True
elif FLAGS.hint_mode == 'none':
    encode_hints = False
    decode_hints = False
else:
    raise ValueError(
        'Hint mode not in {encoded_decoded, decoded_only, none}.')

train_lengths = [int(x) for x in FLAGS.train_lengths]

rng = np.random.RandomState(FLAGS.seed)
rng_key = jax.random.PRNGKey(rng.randint(2**32))

2023-03-27 16:36:55,866 - jax._src.lib.xla_bridge - INFO - Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
2023-03-27 16:36:56,271 - jax._src.lib.xla_bridge - INFO - Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA Host Interpreter
2023-03-27 16:36:56,272 - jax._src.lib.xla_bridge - INFO - Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
2023-03-27 16:36:56,273 - jax._src.lib.xla_bridge - INFO - Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.


In [6]:
FLAGS.random_pos = False
# FLAGS.algorithms = ["insertion_sort"]

# Create samplers
(train_samplers,
 val_samplers, val_sample_counts,
 test_samplers, test_sample_counts,
 spec_list) = create_samplers(rng, train_lengths)

2023-03-27 16:36:57.743414: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-03-27 16:36:57,748 - root - INFO - Creating samplers for algo binary_search
2023-03-27 16:36:57.743472: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-03-27 16:36:57,749 - absl - WARNING - Ignoring kwargs {'length_needle', 'p'} when building sampler class <class 'clrs._src.samplers.SearchSampler'>
2023-03-27 16:36:57,750 - absl - WARNING - Sampling dataset on-the-fly, unlimited samples.
2023-

Not! using randomized pos for split train
Not! using randomized pos for split train


2023-03-27 16:36:58,227 - absl - WARNING - Ignoring kwargs {'length_needle', 'p'} when building sampler class <class 'clrs._src.samplers.SearchSampler'>
2023-03-27 16:36:58,228 - absl - WARNING - Sampling dataset on-the-fly, unlimited samples.
2023-03-27 16:36:58,401 - absl - WARNING - Ignoring kwargs {'length_needle', 'p'} when building sampler class <class 'clrs._src.samplers.SearchSampler'>
2023-03-27 16:36:58,402 - absl - WARNING - Sampling dataset on-the-fly, unlimited samples.


Not! using randomized pos for split train
Not! using randomized pos for split train


2023-03-27 16:36:58,572 - absl - WARNING - Ignoring kwargs {'length_needle', 'p'} when building sampler class <class 'clrs._src.samplers.SearchSampler'>
2023-03-27 16:36:58,573 - absl - INFO - Creating a dataset with 4096 samples.


Not! using randomized pos for split train


2023-03-27 16:36:58,951 - absl - INFO - 1000 samples created
2023-03-27 16:36:59,142 - absl - INFO - 2000 samples created
2023-03-27 16:36:59,332 - absl - INFO - 3000 samples created
2023-03-27 16:36:59,580 - absl - INFO - 4000 samples created
2023-03-27 16:36:59,717 - root - INFO - Dataset found at /tmp/CLRS30/CLRS30_v1.0.0. Skipping download.
2023-03-27 16:36:59,721 - absl - INFO - Load dataset info from /tmp/CLRS30/CLRS30_v1.0.0/clrs_dataset/binary_search_test/1.0.0
2023-03-27 16:36:59,724 - absl - INFO - Load dataset info from /tmp/CLRS30/CLRS30_v1.0.0/clrs_dataset/binary_search_test/1.0.0
2023-03-27 16:36:59,726 - absl - INFO - Reusing dataset clrs_dataset (/tmp/CLRS30/CLRS30_v1.0.0/clrs_dataset/binary_search_test/1.0.0)
2023-03-27 16:36:59,727 - absl - INFO - Constructing tf.data.Dataset clrs_dataset for split test, from /tmp/CLRS30/CLRS30_v1.0.0/clrs_dataset/binary_search_test/1.0.0


Not! using randomized pos for split val
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-03-27 16:36:59,830 - tensorflow - WARNING - From /home/kausta/miniconda3/envs/pysr/lib/python3.8/site-packages/tensorflow/python/autograph/pyct/static_analysis/liveness.py:83: Analyzer.lamba_check (from tensorflow.python.autograph.pyct.static_analysis.liveness) is deprecated and will be removed after 2023-09-23.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Not! using randomized pos for split test


In [7]:
FLAGS.hidden_size = 128 # 32
FLAGS.msg_size = 32 # 32

FLAGS.train_steps = 10000 # 1000
FLAGS.eval_every = 50
FLAGS.test_every = 500

FLAGS.l1_weight = 0.01 # 0.001 

# l1_weight_fn_mult_start = 1
# l1_weight_fn_mult_end = 10
# l1_weight_fn = lambda step: FLAGS.l1_weight * (l1_weight_fn_mult_start + step / FLAGS.train_steps * (l1_weight_fn_mult_end - l1_weight_fn_mult_start))

l1_weight_fn = lambda step: FLAGS.l1_weight

In [8]:
# FLAGS.hidden_size = 8
# FLAGS.algorithms = ['dijkstra']

In [9]:
processor_factory = model.get_processor_factory(
    FLAGS.processor_type,
    use_ln=FLAGS.use_ln,
    nb_triplet_fts=FLAGS.nb_triplet_fts,
    nb_heads=FLAGS.nb_heads
)
model_params = dict(
    processor_factory=processor_factory,
    hidden_dim=FLAGS.hidden_size,
    msg_dim=FLAGS.msg_size,
    encode_hints=encode_hints,
    decode_hints=decode_hints,
    encoder_init=FLAGS.encoder_init,
    use_lstm=FLAGS.use_lstm,
    learning_rate=FLAGS.learning_rate,
    grad_clip_max_norm=FLAGS.grad_clip_max_norm,
    checkpoint_path=FLAGS.checkpoint_path,
    freeze_processor=FLAGS.freeze_processor,
    dropout_prob=FLAGS.dropout_prob,
    hint_teacher_forcing=FLAGS.hint_teacher_forcing,
    hint_repred_mode=FLAGS.hint_repred_mode,
    nb_msg_passing_steps=FLAGS.nb_msg_passing_steps,
    l1_weight=FLAGS.l1_weight
)

eval_model = model.BaselineMsgModel(
    spec=spec_list,
    dummy_trajectory=[next(t) for t in val_samplers],
    **model_params
)
# # we will never used chunked training
# if FLAGS.chunked_training:
#     train_model = clrs.models.BaselineModelChunked(
#         spec=spec_list,
#         dummy_trajectory=[next(t) for t in train_samplers],
#         **model_params
#     )
# else:
#     train_model = eval_model
train_model = eval_model

In [10]:
# Training loop.
best_score = -1.0
current_train_items = [0] * len(FLAGS.algorithms)
step = 0
next_eval = 0
# Make sure scores improve on first step, but not overcome best score
# until all algos have had at least one evaluation.
val_scores = [-99999.9] * len(FLAGS.algorithms)
length_idx = 0

while step < FLAGS.train_steps:
    feedback_list = [next(t) for t in train_samplers]

    # Initialize model.
    if step == 0:
        all_features = [f.features for f in feedback_list]
        if FLAGS.chunked_training:
            # We need to initialize the model with samples of all lengths for
            # all algorithms. Also, we need to make sure that the order of these
            # sample sizes is the same as the order of the actual training sizes.
            all_length_features = [all_features] + [
                [next(t).features for t in train_samplers]
                for _ in range(len(train_lengths))]
            train_model.init(all_length_features[:-1], FLAGS.seed + 1)
        else:
            train_model.init(all_features, FLAGS.seed + 1)

    train_model.l1_weight = l1_weight_fn(step)
    # Training step.
    for algo_idx in range(len(train_samplers)):
        feedback = feedback_list[algo_idx]
        rng_key, new_rng_key = jax.random.split(rng_key)
        if FLAGS.chunked_training:
            # In chunked training, we must indicate which training length we are
            # using, so the model uses the correct state.
            length_and_algo_idx = (length_idx, algo_idx)
        else:
            # In non-chunked training, all training lengths can be treated equally,
            # since there is no state to maintain between batches.
            length_and_algo_idx = algo_idx
        cur_loss = train_model.feedback(
            rng_key, feedback, length_and_algo_idx)
        rng_key = new_rng_key

        if FLAGS.chunked_training:
            examples_in_chunk = np.sum(feedback.features.is_last).item()
        else:
            examples_in_chunk = len(feedback.features.lengths)
        current_train_items[algo_idx] += examples_in_chunk
        if step % 50 == 0:
            logging.info('Algo %s step %i current loss %f, current_train_items %i, l1_weight %f.',
                        FLAGS.algorithms[algo_idx], step,
                        cur_loss, current_train_items[algo_idx], train_model.l1_weight)

    # Periodically evaluate model
    if step >= next_eval:
        eval_model.params = train_model.params
        for algo_idx in range(len(train_samplers)):
            common_extras = {'examples_seen': current_train_items[algo_idx],
                             'step': step,
                             'algorithm': FLAGS.algorithms[algo_idx]}

            # Validation info.
            new_rng_key, rng_key = jax.random.split(rng_key)
            val_stats = collect_and_eval(
                val_samplers[algo_idx],
                functools.partial(eval_model.predict,
                                  algorithm_index=algo_idx),
                val_sample_counts[algo_idx],
                new_rng_key,
                extras=common_extras)
            logging.info('(val) algo %s step %d: %s',
                         FLAGS.algorithms[algo_idx], step, val_stats)
            val_scores[algo_idx] = val_stats['score']

        next_eval += FLAGS.eval_every

        # If best total score, update best checkpoint.
        # Also save a best checkpoint on the first step.
        msg = (f'best avg val score was '
               f'{best_score/len(FLAGS.algorithms):.3f}, '
               f'current avg val score is {np.mean(val_scores):.3f}, '
               f'val scores are: ')
        msg += ', '.join(
            ['%s: %.3f' % (x, y) for (x, y) in zip(FLAGS.algorithms, val_scores)])
        if (sum(val_scores) > best_score) or step == 0:
            best_score = sum(val_scores)
            logging.info('Checkpointing best model, %s', msg)
            train_model.save_model('best.pkl')
        else:
            logging.info('Not saving new best model, %s', msg)

    step += 1
    length_idx = (length_idx + 1) % len(train_lengths)

logging.info('Checkpointing final model, %s', msg)
train_model.save_model('final.pkl')

logging.info('Restoring best model from checkpoint...')
eval_model.restore_model('best.pkl', only_load_processor=False)
# logging.info('Restoring final model from checkpoint...')
# eval_model.restore_model('final.pkl', only_load_processor=False)

2023-03-27 16:37:24,982 - root - INFO - Algo binary_search step 0 current loss 7.195256, current_train_items 32, l1_weight 0.010000.
2023-03-27 16:37:26,590 - root - INFO - (val) algo binary_search step 0: {'return': 0.12744140625, 'score': 0.12744140625, 'examples_seen': 32, 'step': 0, 'algorithm': 'binary_search'}
2023-03-27 16:37:26,591 - root - INFO - Checkpointing best model, best avg val score was -1.000, current avg val score is 0.127, val scores are: binary_search: 0.127
2023-03-27 16:37:59,847 - root - INFO - Algo binary_search step 50 current loss 3.121903, current_train_items 1632, l1_weight 0.010000.
2023-03-27 16:38:00,281 - root - INFO - (val) algo binary_search step 50: {'return': 0.67333984375, 'score': 0.67333984375, 'examples_seen': 1632, 'step': 50, 'algorithm': 'binary_search'}
2023-03-27 16:38:00,282 - root - INFO - Checkpointing best model, best avg val score was 0.127, current avg val score is 0.673, val scores are: binary_search: 0.673
2023-03-27 16:38:01,149 - 

In [11]:
import pickle

def restore_model(model, file_name):
    """Restore model from `file_name`."""
    with open(file_name, 'rb') as f:
        restored_state = pickle.load(f)
        restored_params = restored_state['params']
        model.params = hk.data_structures.merge(restored_params)
        model.opt_state = restored_state['opt_state']

def save_model(model, file_name):
    """Save model (processor weights only) to `file_name`."""
    to_save = {'params': model.params, 'opt_state': model.opt_state}
    with open(file_name, 'wb') as f:
        pickle.dump(to_save, f)

In [12]:
# save_model(eval_model, 'eval_model_1e-3.pkl')

In [13]:
# restore_model(eval_model, 'eval_model_asdf.pkl')

In [14]:
algo_idx = 0
common_extras = {}

new_rng_key, rng_key = jax.random.split(rng_key)
val_stats = collect_and_eval(
    val_samplers[algo_idx],
    functools.partial(eval_model.predict, algorithm_index=algo_idx),
    val_sample_counts[algo_idx],
    new_rng_key,
    extras=common_extras)

print(val_stats)

{'return': 0.88671875, 'score': 0.88671875}


In [15]:
new_rng_key, rng_key = jax.random.split(rng_key)
test_stats = collect_and_eval(
    test_samplers[algo_idx],
    functools.partial(eval_model.predict, algorithm_index=algo_idx),
    test_sample_counts[algo_idx],
    new_rng_key,
    extras=common_extras)

print(test_stats)

{'return': 0.5390625, 'score': 0.5390625}


In [16]:
feedback = next(train_samplers[0])

In [17]:
list(filter(lambda x: x[:2] != '__', dir(feedback)))

['_asdict',
 '_field_defaults',
 '_fields',
 '_fields_defaults',
 '_make',
 '_replace',
 'count',
 'features',
 'index',
 'outputs']

In [18]:
list(filter(lambda x: x[:2] != '__', dir(feedback.features)))

['_asdict',
 '_field_defaults',
 '_fields',
 '_fields_defaults',
 '_make',
 '_replace',
 'count',
 'hints',
 'index',
 'inputs',
 'lengths']

In [19]:
feedback.features

Features(inputs=(DataPoint(name="pos",	location=node,	type=scalar,	data=Array(32, 4)), DataPoint(name="key",	location=node,	type=scalar,	data=Array(32, 4)), DataPoint(name="target",	location=graph,	type=scalar,	data=Array(32,)), DataPoint(name="pred",	location=node,	type=pointer,	data=Array(32, 4))), hints=(DataPoint(name="low",	location=node,	type=mask_one,	data=Array(3, 32, 4)), DataPoint(name="high",	location=node,	type=mask_one,	data=Array(3, 32, 4)), DataPoint(name="mid",	location=node,	type=mask_one,	data=Array(3, 32, 4))), lengths=array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]))

In [20]:
feedback.features.inputs

(DataPoint(name="pos",	location=node,	type=scalar,	data=Array(32, 4)),
 DataPoint(name="key",	location=node,	type=scalar,	data=Array(32, 4)),
 DataPoint(name="target",	location=graph,	type=scalar,	data=Array(32,)),
 DataPoint(name="pred",	location=node,	type=pointer,	data=Array(32, 4)))

In [21]:
feedback.features.inputs[0].data

array([[0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.  , 0.25, 0.5 , 0.75],
       [0.

In [22]:
feedback.features.inputs[1].data

array([[0.02520887, 0.34267156, 0.73365599, 0.8296627 ],
       [0.2815863 , 0.4715894 , 0.47860625, 0.48026793],
       [0.39097453, 0.47543256, 0.57908949, 0.70142573],
       [0.30469259, 0.42536476, 0.53733447, 0.57030581],
       [0.21131138, 0.40548309, 0.75526293, 0.99791545],
       [0.02907753, 0.18829309, 0.25950104, 0.71756621],
       [0.37008792, 0.50818862, 0.70534563, 0.8590399 ],
       [0.18123752, 0.31062654, 0.42587453, 0.51728185],
       [0.17784155, 0.36243278, 0.53391556, 0.64810164],
       [0.11175705, 0.25690464, 0.30079149, 0.83954411],
       [0.0177951 , 0.05114759, 0.7110521 , 0.72883802],
       [0.62460559, 0.75709726, 0.92234513, 0.94757189],
       [0.06444846, 0.077098  , 0.34084127, 0.36992203],
       [0.22745867, 0.23235493, 0.64552243, 0.70196529],
       [0.02107851, 0.19214855, 0.90481048, 0.99302876],
       [0.01860355, 0.39062096, 0.56357491, 0.69268914],
       [0.10956793, 0.1245223 , 0.54496254, 0.64379351],
       [0.02517611, 0.36283305,

In [23]:
feedback.features.inputs[2].data

array([0.82062526, 0.52817175, 0.71065433, 0.3316224 , 0.90621436,
       0.51697819, 0.03612513, 0.07783235, 0.75587324, 0.06289266,
       0.36138149, 0.56755967, 0.60333232, 0.00822275, 0.06009184,
       0.78350037, 0.95836797, 0.25672611, 0.09071109, 0.63983783,
       0.2822263 , 0.40492029, 0.2021623 , 0.37730146, 0.61462071,
       0.71113094, 0.62929972, 0.03444254, 0.93701735, 0.4842491 ,
       0.08684329, 0.46058426])

In [24]:
feedback.features.inputs[3].data

array([[0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.]])

In [25]:
A = feedback.features.inputs[2].data
adj = feedback.features.inputs[3].data

In [26]:
jnp.where(jnp.expand_dims((A != 0), 1) & (adj == 0))

(Array([ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,
         8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16,
        17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25,
        25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31], dtype=int32),
 Array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],      dtype=int32))

In [27]:
feedback.features.hints

(DataPoint(name="low",	location=node,	type=mask_one,	data=Array(3, 32, 4)),
 DataPoint(name="high",	location=node,	type=mask_one,	data=Array(3, 32, 4)),
 DataPoint(name="mid",	location=node,	type=mask_one,	data=Array(3, 32, 4)))

In [28]:
np.argmin(feedback.features.inputs[0].data, axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [29]:
np.where(feedback.outputs[0].data == 1)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]),
 array([3, 3, 3, 1, 3, 3, 0, 0, 3, 0, 2, 0, 3, 0, 1, 3, 3, 1, 1, 3, 0, 3,
        1, 1, 3, 3, 1, 0, 3, 2, 0, 0]))

In [30]:
test_sample_counts[0]

2048

In [31]:
feedback.features

Features(inputs=(DataPoint(name="pos",	location=node,	type=scalar,	data=Array(32, 4)), DataPoint(name="key",	location=node,	type=scalar,	data=Array(32, 4)), DataPoint(name="target",	location=graph,	type=scalar,	data=Array(32,)), DataPoint(name="pred",	location=node,	type=pointer,	data=Array(32, 4))), hints=(DataPoint(name="low",	location=node,	type=mask_one,	data=Array(3, 32, 4)), DataPoint(name="high",	location=node,	type=mask_one,	data=Array(3, 32, 4)), DataPoint(name="mid",	location=node,	type=mask_one,	data=Array(3, 32, 4))), lengths=array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]))

In [33]:
new_rng_key, rng_key = jax.random.split(rng_key)
batched_msgs = get_msgs(
    test_samplers[0],
    functools.partial(eval_model.predict, algorithm_index=0),
#     test_sample_counts[0],  # EXPLODING MEMORY LOL
    4*32*4,
    new_rng_key,
    # For binary search: mid1 == 1 or mid2 == 1
    criteria=lambda x: np.bitwise_or(x[:, -1] == 1, x[:, -2] == 1),
    # criteria=None,
    sample_prob=1)

batched_msgs.shape

(32, 6, 64, 64, 32) 32 512 12
Thresholding: 24192 of 786432
Criteria thresholding: 128 of 786432
Random sampling: 128 of 786432
(32, 6, 64, 64, 32) 32 512 12
Thresholding: 24192 of 786432
Criteria thresholding: 128 of 786432
Random sampling: 128 of 786432
(32, 6, 64, 64, 32) 32 512 12
Thresholding: 24192 of 786432
Criteria thresholding: 128 of 786432
Random sampling: 128 of 786432
(32, 6, 64, 64, 32) 32 512 12
Thresholding: 24192 of 786432
Criteria thresholding: 128 of 786432
Random sampling: 128 of 786432
(32, 6, 64, 64, 32) 32 512 12
Thresholding: 24192 of 786432
Criteria thresholding: 128 of 786432
Random sampling: 128 of 786432
(32, 6, 64, 64, 32) 32 512 12
Thresholding: 24192 of 786432
Criteria thresholding: 128 of 786432
Random sampling: 128 of 786432
(32, 6, 64, 64, 32) 32 512 12
Thresholding: 24192 of 786432
Criteria thresholding: 128 of 786432
Random sampling: 128 of 786432
(32, 6, 64, 64, 32) 32 512 12
Thresholding: 24192 of 786432
Criteria thresholding: 128 of 786432
Random 

(2048, 556)

In [34]:
jnp.sum(jnp.linalg.norm(batched_msgs[:,:32],axis=1) < 0.01)

Array(0, dtype=int32)

In [35]:
jnp.sum(jnp.max(jnp.abs(batched_msgs[:,:32]),axis=1) < 0.001)

Array(0, dtype=int32)

In [36]:
batched_msgs[0][:32], batched_msgs[1][:32], batched_msgs[2][:32]

(array([ 0.01300676, -0.00516211,  0.04877086,  0.05424519,  0.02105969,
         0.01826444, -0.01051411, -0.0716765 ,  0.03990274,  0.07754934,
        -0.02410381, -0.05027805,  0.04341411,  0.04791321, -0.00606619,
        -0.03409918, -0.00920935, -0.00709733,  0.07736857,  0.01340538,
        -0.84274709,  0.05170161, -0.01429862, -0.5576036 , -0.01667661,
        -0.012039  ,  0.08439622, -0.06144607,  0.00974206,  0.00482731,
        -0.00879854,  0.01555658]),
 array([ 0.23687603,  0.01969739, -0.03851569,  0.22293663, -0.05194453,
        -0.18434471, -0.00436173,  0.03789113, -0.090555  , -0.07392705,
         0.03429982,  0.01792889,  0.10030431,  0.01679242,  0.0422462 ,
        -0.17402378, -0.08953831, -0.11886487, -0.07830518, -0.01434487,
        -0.84145832, -0.02248025, -0.04409573, -0.53596944,  0.09152045,
        -0.07621835, -0.07672852,  0.1089345 , -0.22144185,  0.34828785,
         0.04142797, -0.06552748]),
 array([ 2.83360243e-01,  2.20238939e-02, -2.1438021

In [37]:
file_name = 'binary_search_val_msgs.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(batched_msgs, f)

In [38]:
import sys
sys.getsizeof(batched_msgs)

9109632

In [39]:
test_sample_counts[0]

2048

In [40]:
sys.getsizeof(batched_msgs) / 1000000

9.109632